In [1]:
import os
import re
import glob
import uproot
import numpy  as np
import pandas as pd

from os.path import expandvars, join, basename

import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
def read_time_window_data(filename):

    columns = ["Event", "ntws", "cluster", "t0", "t1", "pt", "px", "py", "pz", "nsub", "t", "G"]
    df = pd.DataFrame(columns=columns)
    
    f = uproot.open(filename)
    t = f["fiTQun"]
    fqntwnd          = t["fqntwnd"]         .array().to_numpy()-1
    fqtwnd_iclstr    = t["fqtwnd_iclstr"]   .array().to_numpy()
    fqtwnd_npeak     = t["fqtwnd_npeak"]    .array().to_numpy()
    fqtwnd_prftt0    = t["fqtwnd_prftt0"]   .array().to_numpy()
    fqtwnd_prftpos   = t["fqtwnd_prftpos"]  .array().to_numpy()
    fqtwnd           = t["fqtwnd"]          .array().to_numpy()
    fqtwnd_peakt0    = t["fqtwnd_peakt0"]   .array().to_numpy()
    fqtwnd_peakiness = t["fqtwnd_peakiness"].array().to_numpy()

    nevents = len(fqntwnd)

    for ev, event in enumerate(range(nevents)):
        nc = fqntwnd[ev]
        for cluster in fqtwnd_iclstr[ev]:
            ts   = fqtwnd        [ev, cluster]
            pt   = fqtwnd_prftt0 [ev, cluster]
            ppos = fqtwnd_prftpos[ev, cluster]
            nsub = fqtwnd_npeak  [ev, cluster]
            for subp in range(nsub):
                t = fqtwnd_peakt0   [ev, cluster, subp]
                G = fqtwnd_peakiness[ev, cluster, subp]
                df.loc[len(df)] = (event, nc, cluster, *ts, pt, *ppos, nsub, t, G)
    f.close()
    return df


def read_subevent_data(filename):

    columns = ["Event", "nsub", "tw", "peak", "N", "Q", "Q0R", "nll0R", "n50", "q50"]
    df = pd.DataFrame(columns=columns)
    
    f = uproot.open(filename)
    t = f["fiTQun"]

    fqnse     = t["fqnse"]    .array().to_numpy()
    fqitwnd   = t["fqitwnd"]  .array().to_numpy()
    fqipeak   = t["fqipeak"]  .array().to_numpy()
    fqnhitpmt = t["fqnhitpmt"].array().to_numpy()
    fqtotq    = t["fqtotq"]   .array().to_numpy()
    fq0rtotmu = t["fq0rtotmu"].array().to_numpy()
    fq0rnll   = t["fq0rnll"]  .array().to_numpy()
    fqn50     = t["fqn50"]    .array().to_numpy()
    fqq50     = t["fqq50"]    .array().to_numpy()

    nevents = len(fqnse)

    for ev, event in enumerate(range(nevents)):
        nsub = fqnse[ev]
        for subp in range(nsub):
            tw    = fqitwnd  [ev, subp]
            peak  = fqipeak  [ev, subp]
            N     = fqnhitpmt[ev, subp]
            Q     = fqtotq   [ev, subp]
            Q0R   = fq0rtotmu[ev, subp]
            nll0R = fq0rnll  [ev, subp]
            n50   = fqn50    [ev, subp]
            q50   = fqq50    [ev, subp]
            df.loc[len(df)] = (event, nsub, tw, peak, N, Q, Q0R, nll0R, n50, q50)
    f.close()
    return df


def read_1Ring_data(filename, pids, event_counter=0):

    columns = ["Event", "peak", "pid", "pc", "p", "t", "x", "y", "z", "theta", "phi", "Q1R", "nll1R", "L", "Eloss"]
    df = pd.DataFrame(columns=columns)
    
    f = uproot.open(filename)
    t = f["fiTQun"]

    fqnse   = t["fqnse"]  .array()
    fqipeak = t["fqipeak"].array()
    nevents = len(fqnse)

    fq1rpcflg = t["fq1rpcflg"].array()
    fq1rtotmu = t["fq1rtotmu"].array()
    fq1rnll   = t["fq1rnll"]  .array()
    fq1rmom   = t["fq1rmom"]  .array()
    fq1rt0    = t["fq1rt0"]   .array()
    fq1rpos   = t["fq1rpos"]  .array()
    fq1rdir   = t["fq1rdir"]  .array()
    fq1rdconv = t["fq1rdconv"].array()
    fq1reloss = t["fq1reloss"].array()

    for ev in range(nevents):
        event = event_counter + ev
        nsub = fqnse[ev]
        for subp in range(nsub):
            peak = fqipeak [ev, subp]
            for pid in pids:
                pc    = fq1rpcflg[ev, subp, pid]
                mom   = fq1rmom  [ev, subp, pid]
                t     = fq1rt0   [ev, subp, pid]
                pos   = fq1rpos  [ev, subp, pid]
                dir   = fq1rdir  [ev, subp, pid]
                Q1R   = fq1rtotmu[ev, subp, pid]
                nll1R = fq1rnll  [ev, subp, pid]
                L     = fq1rdconv[ev, subp, pid]
                Eloss = fq1reloss[ev, subp, pid]

                theta = np.arccos(dir[2])
                phi   = np.arctan2(dir[1], dir[0])
                if (np.sign(phi)<0): phi += 2.*np.pi

                df.loc[len(df)] = (event, peak, pid, pc, mom, t, *pos, theta, phi, Q1R, nll1R, L, Eloss)
    f.close()
    return df


In [3]:
filename = expandvars("$HOME/Software/WCSimFQTuningTools/fiTQun/local/out_fitqun.root")

In [4]:
read_1Ring_data(filename, [1])

,Event,peak,pid,pc,p,t,x,y,z,theta,phi,Q1R,nll1R,L,Eloss
0,0.0,0.0,1.0,1.0,375.418701,945.431274,21.514425,-6.010224,6.850032,1.669140,0.159651,2391.531494,3617.336426,0.0,0.0
1,1.0,0.0,1.0,0.0,216.901520,944.766052,4.007757,-3.377138,-1.750460,1.547567,0.038189,1627.645264,3619.736572,0.0,0.0
2,2.0,0.0,1.0,0.0,233.778763,944.982422,3.869364,0.544117,0.919938,1.555857,0.045251,1790.810181,3714.285645,0.0,0.0
3,3.0,0.0,1.0,1.0,359.616333,944.711975,3.162289,-4.636479,6.670578,1.630562,0.060294,2306.793457,4007.422852,0.0,0.0
4,4.0,0.0,1.0,1.0,370.921143,944.902283,6.639068,4.997871,-5.939883,1.558107,6.271872,2338.393066,4134.313477,0.0,0.0
5,5.0,0.0,1.0,1.0,375.836334,945.294800,17.632402,-0.646395,1.269168,1.574558,0.016764,2376.602783,3846.232422,0.0,0.0
6,6.0,0.0,1.0,1.0,392.156464,945.158691,10.011331,3.802654,2.626206,1.525625,6.243000,2508.623779,4128.187500,0.0,0.0
7,7.0,0.0,1.0,1.0,339.307190,945.122925,14.686755,2.065694,-2.728102,1.567283,6.248220,2148.244629,3780.880127,0.0,0.0
8,8.0,0.0,1.0,0.0,220.039810,945.168213,5.949346,3.377123,4.439396,1.567113,0.046597,1675.114990,3767.039307,0.0,0.0
9,9.0,0.0,1.0,0.0,140.488556,944.392944,-4.504966,-4.891351,-10.591654,1.444108,0.045461,1088.346680,3065.389404,0.0,0.0
